In [1]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="ticks")
wine= pd.read_csv("whitewine-training-ds6040.csv")
wine_test= pd.read_csv("whitewine-testing-ds6040.csv")

In [2]:
class LDA():
    def __init__(self, dataset, class_var, priors = None):
        n_class = len(dataset[class_var].unique())
        if priors is None:
            priors = np.repeat(1/n_class, n_class)
        self.priors =  np.asarray(priors)
        self.means = dataset.groupby(class_var).mean()
        self.sigma = dataset.cov()
        self.class_var = class_var
        self.training_data = dataset
    def predict_probs(self, data = None):
        if data is None:
            data = self.training_data
        data_temp = data.drop(self.class_var, axis = 1)
        dens_list = []
        col_names = []
        for ind, row in self.means.iterrows():
            col_names.append(ind)
            dens_list.append(multivariate_normal.pdf(data_temp, mean = np.asarray(row), cov = self.sigma))
        dens_list = pd.DataFrame(np.transpose(np.vstack(dens_list)),columns= col_names)
        dens_list = dens_list.mul(self.priors, axis=1)
        dens_list = dens_list.div(dens_list.sum(axis=1), axis=0)
        dens_list['True Class'] = data[self.class_var]
        return dens_list 
    def predict_MAP(self, data = None):        
        if data is None:
            data = self.training_data
        dens_list = self.predict_probs(data).drop('True Class', axis = 1)
        map_list = dens_list.idxmax(axis = 1)
        maps = {'MAP Class': map_list}
        maps = pd.DataFrame(maps)
        maps['True Class'] = data[self.class_var]
        return maps
    def misclass_rate(self, data = None):
        if data is None:
            data = self.training_data
        maps = self.predict_MAP(data = data)
        
        
        maps['Mis_class'] = maps['MAP Class']  == maps['True Class']
        
        mis_class =  1 - maps['Mis_class'].mean()
        
        return mis_class
    def misclass_xtabs(self, data = None):
        if data is None:
            data = self.training_data
        maps = self.predict_MAP(data = data)
        
        xtabs = pd.crosstab(maps['MAP Class'], maps['True Class'])        
        return xtabs
    def misclass_pairplot(self, data = None):
        if data is None:
            data = self.training_data
        maps = self.predict_MAP(data = data)
        temp_dat = data.copy(deep = True)
        temp_dat['Mis-Classified'] = maps['MAP Class']  != maps['True Class']
        plot = sns.pairplot(temp_dat,hue="Mis-Classified", height = 1.5, aspect = 1.5)   
        return plot

# Question 1

## (1a) Non-informative (flat) priors on wine quality

In [3]:
LDA_wine_flat = LDA(wine, 'wine_quality') #defualt is uniform prob for priors

### Misclassification against training set

In [4]:
LDA_wine_flat.misclass_rate()

0.49496538703587156

It looks like the training set misclassification is quite high at 49.5%. 

In [5]:
LDA_wine_flat.misclass_xtabs()

True Class,A,C,F
MAP Class,,,
A,69,687,62
C,22,800,202
F,9,591,736


### Misclassification against test set

In [6]:
LDA_wine_flat.misclass_rate(data= wine_test)

0.48546511627906974

In [7]:
LDA_wine_flat.misclass_xtabs(data= wine_test)

True Class,A,C,F
MAP Class,,,
A,56,348,43
C,15,381,149
F,9,271,448


The misclassification rate is very high at 49.5% against the training set and 48.5% against the test set. It's interesting to note that the rate against the test set is lower than the one against the training set.

### (1b) Priors that reflect the observed proportion of wines at different quality levels.

In [8]:
wine_quality_prop = wine['wine_quality'].value_counts(normalize=True) 
wine_quality_prop 

C    0.653870
F    0.314663
A    0.031466
Name: wine_quality, dtype: float64

In [9]:
priors_observed= [wine_quality_prop[2], wine_quality_prop[0], wine_quality_prop[1] ]

In [10]:
LDA_wine_observed = LDA(wine, 'wine_quality', priors_observed) 

### Misclassification against training set

In [11]:
LDA_wine_observed.misclass_rate()

0.28099433606041535

In [12]:
LDA_wine_observed.misclass_xtabs()

True Class,A,C,F
MAP Class,,,
A,0,0,1
C,100,1913,627
F,0,165,372


### Misclassification against test set

In [13]:
LDA_wine_observed.misclass_rate(data=wine_test)

0.32732558139534884

In [14]:
LDA_wine_observed.misclass_xtabs(data=wine_test)

True Class,A,C,F
MAP Class,,,
C,79,923,406
F,1,77,234


The misclassification rate when using the observed proportion for the prior was 28% against the training set and 32.7% against the test set. It was interesting to note that there was only one prediction for wine quality A in the training set and no predictions for it in the test set. This is undestandable as the prior for quality A was only around 3%.

### (1c) Priors that reflect the notion that most wines are awful, some wines are average, and few wines are good 

In [15]:
LDA_wine_1c = LDA(wine, 'wine_quality', [.10, .25, .65])

### Misclassification against training set

In [16]:
LDA_wine_1c.misclass_rate()

0.5245437382001259

In [17]:
LDA_wine_1c.misclass_xtabs()

True Class,A,C,F
MAP Class,,,
A,28,137,4
C,34,540,53
F,38,1401,943


### Misclassification against test set

In [18]:
LDA_wine_1c.misclass_rate(data= wine_test)

0.4767441860465116

In [19]:
LDA_wine_1c.misclass_xtabs(data = wine_test)

True Class,A,C,F
MAP Class,,,
A,20,79,2
C,29,269,27
F,31,652,611


Similar to the non-informative priors, these predictoins also resulted in very high misclassification rates. The rates were 52.5% against the training set and 47.7% against the test set. These priors seem to be no better than a non-informative prior and much worse than priors based on the observed proportion.

### (1d) Priors that somebody with terrible taste in wine would use (i.e. most wines are good, few wines are bad or average)

In [20]:
LDA_wine_1d = LDA(wine, 'wine_quality', [.6, .2, .2])

### Misclassification against training set

In [21]:
LDA_wine_1d.misclass_rate()

0.7407174323473883

In [22]:
LDA_wine_1d.misclass_xtabs()

True Class,A,C,F
MAP Class,,,
A,87,1537,330
C,4,94,27
F,9,447,643


### Misclassification against training set

In [23]:
LDA_wine_1d.misclass_rate(data= wine_test)

0.7203488372093023

In [24]:
LDA_wine_1d.misclass_xtabs(data = wine_test)

True Class,A,C,F
MAP Class,,,
A,72,756,230
C,0,35,36
F,8,209,374


The misclassification rate using these priors is extremeley high at 74.1% against the training set and 72% agianst the test set. Looking at the cross-tabs table, it's evident that the model is over predicting class "A" when in reality, that class is the smallest in both the trainig and test set.

# Question 2

In [25]:
from itertools import combinations

In [26]:
cols_list= wine.columns[:-1]

In [27]:
list_combos = list(combinations(cols_list, 3))

In [28]:
len(list_combos)

165

## (2a)

### Misclassification rate against the training set

In [62]:
missclass_dict= {} #dictionary to hold misclass rate for each combo

In [63]:
for combo in list_combos:
    '''
        iterates through each combo, calculates misclass rate and stores in dictionary
    '''
    combo_list = list(combo)
    combo_list.append('wine_quality')
    combo_df= wine[combo_list]
    LDA_flat = LDA(combo_df, 'wine_quality') #defualt is uniform prob for priors
    misclass_rate = LDA_flat.misclass_rate()
    missclass_dict[combo] = misclass_rate

In [64]:
#sort di
missclass_dict = sorted(missclass_dict.items(), key = lambda kv: kv[1])
missclass_dict = dict(sorted_dict)
missclass_dict


{('volatile.acidity', 'residual.sugar', 'alcohol'): 0.5050346129641283,
 ('volatile.acidity', 'density', 'alcohol'): 0.5106985525487728,
 ('volatile.acidity', 'pH', 'alcohol'): 0.5204531151667715,
 ('volatile.acidity', 'citric.acid', 'alcohol'): 0.5232850849590938,
 ('volatile.acidity', 'free.sulfur.dioxide', 'alcohol'): 0.5232850849590938,
 ('volatile.acidity', 'total.sulfur.dioxide', 'alcohol'): 0.5235997482693517,
 ('volatile.acidity', 'sulphates', 'alcohol'): 0.5242290748898679,
 ('volatile.acidity', 'chlorides', 'alcohol'): 0.5254877281309,
 ('volatile.acidity', 'residual.sugar', 'density'): 0.526431718061674,
 ('fixed.acidity', 'volatile.acidity', 'alcohol'): 0.5280050346129641,
 ('volatile.acidity', 'citric.acid', 'sulphates'): 0.5305223410950284,
 ('volatile.acidity', 'chlorides', 'sulphates'): 0.5380742605412209,
 ('volatile.acidity', 'citric.acid', 'chlorides'): 0.5415355569540592,
 ('volatile.acidity', 'chlorides', 'pH'): 0.5424795468848332,
 ('volatile.acidity', 'chlorides'